In [31]:
import mlflow
import mlflow.sklearn
from mlflow.tracking.client import MlflowClient
import mlflow.pyfunc
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

In [5]:
data = load_boston()
X = pd.DataFrame(data.data)
y = data.target

In [6]:
def train_sklearn_model(X, y):
  model = LinearRegression() 
  model.fit(X, y)
  return model

In [7]:
sql_string = "http://localhost:5000"
mlflow.set_tracking_uri(sql_string)

In [8]:
# Создаем эксперимент
expname = "Boston_exp"
exp_id = mlflow.create_experiment(expname)


In [9]:
print(exp_id)

1


In [10]:
with mlflow.start_run(experiment_id=exp_id, run_name='sk'):
  # Автоматом сохраняет model's parameters, metrics, artifacts
  mlflow.sklearn.autolog()
  train_sklearn_model(X, y)
  run_id_sk = mlflow.active_run().info.run_id

2021/12/31 10:56:50 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [11]:
print(run_id_sk)

b93f7cc7e5fd4726a9ed71a56be33474


In [12]:
# Задаем имя модели
model_name = "model_boston_sk"
 
# Стандартный путь к каталогу с моделями
artifact_path = "model"
model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id_sk, artifact_path=artifact_path)
 
model_details_sk = mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'model_boston_sk'.
2021/12/31 10:57:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: model_boston_sk, version 1
Created version '1' of model 'model_boston_sk'.


In [13]:
# Описание совокупности моделей
client = MlflowClient()
client.update_registered_model(
  name=model_details_sk.name,
  description="Линейная регрессия"
)

<RegisteredModel: creation_timestamp=1640930263815, description='Линейная регрессия', last_updated_timestamp=1640930273959, latest_versions=[<ModelVersion: creation_timestamp=1640930263949, current_stage='None', description='', last_updated_timestamp=1640930263949, name='model_boston_sk', run_id='b93f7cc7e5fd4726a9ed71a56be33474', run_link='', source='mlflow/1/b93f7cc7e5fd4726a9ed71a56be33474/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='model_boston_sk', tags={}>

In [14]:
# Описание версии моделей
client.update_model_version(
  name=model_details_sk.name,
  version=model_details_sk.version,
  description="Линейная регрессия. Версия 1"
)


<ModelVersion: creation_timestamp=1640930263949, current_stage='None', description='Линейная регрессия. Версия 1', last_updated_timestamp=1640930280734, name='model_boston_sk', run_id='b93f7cc7e5fd4726a9ed71a56be33474', run_link='', source='mlflow/1/b93f7cc7e5fd4726a9ed71a56be33474/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [16]:
# Изменяем ТЕГ модели
client.transition_model_version_stage(
  name=model_details_sk.name,
  version=model_details_sk.version,
  stage='Production',
)

<ModelVersion: creation_timestamp=1640930263949, current_stage='Production', description='Линейная регрессия. Версия 1', last_updated_timestamp=1640930958089, name='model_boston_sk', run_id='b93f7cc7e5fd4726a9ed71a56be33474', run_link='', source='mlflow/1/b93f7cc7e5fd4726a9ed71a56be33474/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [20]:
# Смотрим на статус модели по имени
model_version_details = client.get_model_version(
  name=model_details_sk.name,
  version=model_details_sk.version,
)
print("The current model stage is: '{stage}'".format(stage=model_details_sk.current_stage))

The current model stage is: 'None'


In [21]:
latest_version_info = client.get_latest_versions(model_name, stages=["Production"])
latest_production_version = latest_version_info[0].version
print("The latest production version of the model '%s' is '%s'." % (model_name, latest_production_version))

The latest production version of the model 'model_boston_sk' is '1'.


In [22]:
experiments = client.list_experiments()
experiments

[<Experiment: artifact_location='mlflow/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='mlflow/1', experiment_id='1', lifecycle_stage='active', name='Boston_exp', tags={}>]

In [23]:
experiments[-1].experiment_id

'1'

In [24]:
run = client.create_run(experiments[-1].experiment_id)
run


<Run: data=<RunData: metrics={}, params={}, tags={}>, info=<RunInfo: artifact_uri='mlflow/1/464056b50cb84bfaab18220d886cc06c/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='464056b50cb84bfaab18220d886cc06c', run_uuid='464056b50cb84bfaab18220d886cc06c', start_time=1640931512423, status='RUNNING', user_id='unknown'>>

In [25]:
client.list_run_infos(experiments[-1].experiment_id)

[<RunInfo: artifact_uri='mlflow/1/464056b50cb84bfaab18220d886cc06c/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='464056b50cb84bfaab18220d886cc06c', run_uuid='464056b50cb84bfaab18220d886cc06c', start_time=1640931512423, status='RUNNING', user_id='unknown'>,
 <RunInfo: artifact_uri='mlflow/1/b93f7cc7e5fd4726a9ed71a56be33474/artifacts', end_time=1640930232244, experiment_id='1', lifecycle_stage='active', run_id='b93f7cc7e5fd4726a9ed71a56be33474', run_uuid='b93f7cc7e5fd4726a9ed71a56be33474', start_time=1640930210887, status='FINISHED', user_id='pavel'>]

In [28]:
model_version_uri = "models:/{model_name}/1".format(model_name=model_details_sk.name)
 
print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_version_uri))
model_sk = mlflow.pyfunc.load_model(model_version_uri)

Loading registered model version from URI: 'models:/model_boston_sk/1'


In [30]:
y_predict = model_sk.predict(X)
y_predict[:5]

array([30.00384338, 25.02556238, 30.56759672, 28.60703649, 27.94352423])

In [32]:
mean_absolute_error(y,y_predict)

3.270862810900316